In [4]:
import imaplib
import email
from email.header import decode_header
import webbrowser
import os
from bs4 import BeautifulSoup
import re

# account credentials
username = "d177078_0-dev"
password = ")eg2Q@@rFGegElL"
# use your email provider's IMAP server, you can look for your provider's IMAP server on Google
# or check this page: https://www.systoolsgroup.com/imap/
# for office 365, it's this:
imap_server = "imap.1blu.de"


In [2]:
def clean(text):
    # clean text for creating a folder
    return "".join(c if c.isalnum() else "_" for c in text)

In [6]:
# create an IMAP4 class with SSL 
imap = imaplib.IMAP4_SSL(imap_server)
# authenticate
imap.login(username, password)

status, messages = imap.select("INBOX")
# number of top emails to fetch
N = 4
# total number of emails
messages = int(messages[0])
offset = 0
msg = ""
# imap.create("danil")
for i in range(messages-offset, messages-offset-N, -1):
    print(i)
    res, mail = imap.fetch(str(i), '(RFC822)')
    # imap.copy(str(i), "danil")
    print(mail[0])
    body = mail[0][1]
    msg = email.message_from_bytes(body)
    print(msg.is_multipart())

    attachement_count = len(msg.get_payload())-1

    if attachement_count > 0:
        for j in range(1, attachement_count+1):
            attachment = msg.get_payload(j).__str__()
            soup = BeautifulSoup(attachment)   
            f = open(f'''alias_mail{i}_attachment{j}.txt''', 'w')
            f.write(soup.get_text())
            f.close()
    
    payload = msg.get_payload(0).__str__()

    soup = BeautifulSoup(payload)
    f = open(f'''alias_mail{i}.eml''', 'w')
    f.write(soup.get_text())
    # f.write(str(body))
    f.close()

imap.close()
imap.logout()

18
(b'18 (RFC822 {5137}', b'Return-Path: <bounce-md_30012989.6303eaba.v1-f37b01e113184e019db7916a082490fc@mandrillapp.com>\r\nDelivered-To: d177078_0-daniel@mb-26.1blu.de\r\nReceived: from ms-10.1blu.de ([172.16.180.5])\r\n\tby mb-26.1blu.de with LMTP\r\n\tid aV+zObvqA2OAeDQAfvvPVA\r\n\t(envelope-from <bounce-md_30012989.6303eaba.v1-f37b01e113184e019db7916a082490fc@mandrillapp.com>)\r\n\tfor <d177078_0-daniel@mb-26.1blu.de>; Mon, 22 Aug 2022 22:44:43 +0200\r\nReceived: from [198.2.186.114] (helo=mail186-114.suw21.mandrillapp.com)\r\n\tby ms-10.1blu.de with esmtps (TLS1.2:ECDHE_RSA_AES_256_GCM_SHA384:256)\r\n\t(Exim 4.90_1)\r\n\t(envelope-from <bounce-md_30012989.6303eaba.v1-f37b01e113184e019db7916a082490fc@mandrillapp.com>)\r\n\tid 1oQEHr-0002hu-Nc\r\n\tfor resell@danielhighfive.de; Mon, 22 Aug 2022 22:44:43 +0200\r\nDKIM-Signature: v=1; a=rsa-sha256; c=relaxed/relaxed; d=alias.org;\r\n\ts=mandrill; t=1661201082; x=1661503482; i=info@alias.org;\r\n\tbh=Af6QMLYSl86Gw24qMUTr/Hu+S4V2sNEJa

('BYE', [b'Logging out'])

In [ ]:
mailbox = imaplib.IMAP4_SSL(imap_server)
# authenticate
mailbox.login(username, password)
status, messages = mailbox.select("INBOX")
N = 4
messages = int(messages[0])
offset = 0
for i in range(messages-offset, messages-offset-N, -1):
    typ, data = mailbox.fetch(str(i), '(RFC822)')
    for response_part in data:
        if isinstance(response_part, tuple):
            msg = email.message_from_bytes(response_part[1])
            if msg['Message-Id']:
                filename = re.sub('[^a-zA-Z0-9_\-\.()\s]+', '', msg['Message-Id'])
                f = open('%s/%s.eml' %(".", filename), 'w')
                f.write(str(data[0][1]))
                f.close()
            else:
                print(f'''No Message-Id for {msg['Subject']}''')
    
mailbox.close()
mailbox.logout()

In [ ]:
# create an IMAP4 class with SSL 
imap = imaplib.IMAP4_SSL(imap_server)
# authenticate
imap.login(username, password)

status, messages = imap.select("INBOX")
# number of top emails to fetch
N = 3
# total number of emails
messages = int(messages[0])

for i in range(messages, messages-N, -1):
    # fetch the email message by ID
    res, msg = imap.fetch(str(i), '(RFC822)')
    for response in msg:
        if isinstance(response, tuple):
            # parse a bytes email into a message object
            msg = email.message_from_bytes(response[1])
            # decode the email subject
            subject, encoding = decode_header(msg["Subject"])[0]
            if isinstance(subject, bytes):
                # if it's a bytes, decode to str
                subject = subject.decode(encoding)
            # decode email sender
            From, encoding = decode_header(msg.get("From"))[0]
            if isinstance(From, bytes):
                From = From.decode(encoding)
            print("Subject:", subject)
            print("From:", From)
            # if the email message is multipart
            if msg.is_multipart():
                # iterate over email parts
                for part in msg.walk():
                    # extract content type of email
                    content_type = part.get_content_type()
                    content_disposition = str(part.get("Content-Disposition"))
                    try:
                        # get the email body
                        body = part.get_payload(decode=True).decode()
                    except:
                        pass
                    if content_type == "text/plain" and "attachment" not in content_disposition:
                        # print text/plain emails and skip attachments
                        print(body)
                    elif "attachment" in content_disposition:
                        # download attachment
                        filename = part.get_filename()
                        if filename:
                            folder_name = clean(subject)
                            if not os.path.isdir(folder_name):
                                # make a folder for this email (named after the subject)
                                os.mkdir(folder_name)
                            filepath = os.path.join(folder_name, filename)
                            # download attachment and save it
                            open(filepath, "wb").write(part.get_payload(decode=True))
            else:
                # extract content type of email
                content_type = msg.get_content_type()
                # get the email body
                body = msg.get_payload(decode=True).decode()
                if content_type == "text/plain":
                    # print only text email parts
                    print(body)
            if content_type == "text/html":
                # if it's HTML, create a new HTML file and open it in browser
                folder_name = clean(subject)
                if not os.path.isdir(folder_name):
                    # make a folder for this email (named after the subject)
                    os.mkdir(folder_name)
                filename = "index.html"
                filepath = os.path.join(folder_name, filename)
                # write the file
                open(filepath, "w").write(body)
                # open in the default browser
                webbrowser.open(filepath)
            print("="*100)
# close the connection and logout
imap.close()
imap.logout()